In [1]:
# Model comparisons

In [2]:
import cv2
from face_alignment import FaceAlignment, LandmarksType
import os
import urllib.request as urlreq

from loreal_poc.dataloaders.loaders import DataLoader300W
from loreal_poc.dataloaders.wrappers import CroppedDataLoader

from loreal_poc.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from loreal_poc.tests.performance import NMEMean
from loreal_poc.tests.base import Test, TestDiff
from loreal_poc.marks.facial_parts import FacialParts

In [3]:
dl = DataLoader300W(dir_path="300W/sample")
chosen_idx = 1
image, ground_truth_landmarks, _ = dl[chosen_idx]

In [4]:
facealignment_model = FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device="cpu", flip_input=False))
opencv_model = OpenCVWrapper()

loading data from : lbfmodel.yaml


In [5]:
# from dataclasses import fields
def report(models, dataloader, tests, facial_parts):
    results = []
    for model in models:
        # for __facial_part in fields(FacialParts):
        #  _facial_part = __facial_part.default
        for facial_part in facial_parts:
            for test_cls in tests:
                test_result = test_cls(metric=NMEMean, threshold=1).run(
                    model=model,
                    dataloader=CroppedDataLoader(dataloader, part=facial_part),
                    dataloader_ref=dl,
                    facial_part=facial_part,
                )
                results.append(test_result.to_dict())
    return results


report = report(
    [opencv_model, facealignment_model],
    dl,
    [TestDiff],
    [FacialParts.bottom_half, FacialParts.upper_half, FacialParts.left_half, FacialParts.right_half],
)

In [ ]:
import pandas as pd

pd.DataFrame(report)

,test_name,metric_name,metric_value,threshold,passed,facial_part,model_name,dataloader_name
0,TestDiff,NME_mean,0.086974,1,True,bottom half,OpenCV,300W cropped on bottom half
1,TestDiff,NME_mean,0.027602,1,True,upper half,OpenCV,300W cropped on upper half
2,TestDiff,NME_mean,0.065393,1,True,left half,OpenCV,300W cropped on left half
3,TestDiff,NME_mean,0.041429,1,True,right half,OpenCV,300W cropped on right half
4,TestDiff,NME_mean,0.071476,1,True,bottom half,FaceAlignment,300W cropped on bottom half
5,TestDiff,NME_mean,0.064830,1,True,upper half,FaceAlignment,300W cropped on upper half
6,TestDiff,NME_mean,0.059586,1,True,left half,FaceAlignment,300W cropped on left half
7,TestDiff,NME_mean,0.076536,1,True,right half,FaceAlignment,300W cropped on right half
